# Compute potential intensity for climate models

```
conda create -n potint -y netcdf4 xarray=0.16.2 numpy numba ipykernel                           # don't add matplotlib after xarray - broke installation
conda activate potint
pip install tcpypi
```

# Sample run to check installation

In [ ]:
# This pyPI script computes PI and associated analyses over the entire sample dataset
# which is from 2004, MERRA2.
#
# Created by Daniel Gilford, PhD (daniel.gilford@rutgers.edu)
# Many thanks to Daniel Rothenberg for his assitance optimizing pyPI
#
# Last updated 8/14/2020
#

# setup
import xarray as xr
import pickle

# load in pyPI modules
from tcpyPI import pi
from tcpyPI.utilities import *

# define the sample data locations
datdir='./data/'
_FN=datdir+'sample_data.nc'
_mdrF=datdir+'mdr.pk1' 
    

def run_sample_dataset(fn, dim='p',CKCD=0.9):
    """ This function calculates PI over the sample dataset using xarray """
    
    # open the sample data file
    ds = xr.open_dataset(fn)
    # calculate PI over the whole data set using the xarray universal function
    result = xr.apply_ufunc(
        pi,
        ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
        kwargs=dict(CKCD=CKCD, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),
        input_core_dims=[
            [], [], ['p', ], ['p', ], ['p', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )

    # store the result in an xarray data structure
    vmax, pmin, ifl, t0, otl = result
    out_ds=xr.Dataset({
        'vmax': vmax, 
        'pmin': pmin,
        'ifl': ifl,
        't0': t0,
        'otl': otl,
        # merge the state data into the same data structure
        'sst': ds.sst,
        't': ds.t,
        'q': ds.q,
        'msl': ds.msl,
        'lsm': ds.lsm,
        })
    
    # add names and units to the structure
    out_ds.vmax.attrs['standard_name'],out_ds.vmax.attrs['units']='Maximum Potential Intensity','m/s'
    out_ds.pmin.attrs['standard_name'],out_ds.pmin.attrs['units']='Minimum Central Pressure','hPa'
    out_ds.ifl.attrs['standard_name']='pyPI Flag'
    out_ds.t0.attrs['standard_name'],out_ds.t0.attrs['units']='Outflow Temperature','K'
    out_ds.otl.attrs['standard_name'],out_ds.otl.attrs['units']='Outflow Temperature Level','hPa'

    # return the output from pi.py as an xarray data structure
    return out_ds

def run_sample_analyses(ds,_mdrF,CKCD=0.9):
    """ This function performs PI analyses over the sample dataset using xarray """

    # load the basins dictionary
    basins = pickle.load( open( _mdrF, "rb" ) )
    
    # calculate PI analyses over the whole data set using the xarray universal function
    efficiency = xr.apply_ufunc(
        pi_efficiency,
        ds['sst']+273.15, ds['t0'],
        input_core_dims=[
            [], [],
        ],
        output_core_dims=[
            [],
        ],
        vectorize=True
    )
    
    diseq = xr.apply_ufunc(
        pi_diseq_resid,
        ds['vmax'], ds['sst']+273.15, ds['t0'],
        kwargs=dict(CKCD=CKCD),
        input_core_dims=[
            [], [], [],
        ],
        output_core_dims=[
            [],
        ],
        vectorize=True
    )
    
    result = xr.apply_ufunc(
        decompose_pi,
        ds['vmax'], ds['sst']+273.15, ds['t0'],
        kwargs=dict(CKCD=CKCD),
        input_core_dims=[
            [], [], [],
        ],
        output_core_dims=[
            [], [], [], [],
        ],
        vectorize=True
    )

    lnpi, lneff, lndiseq, lnCKCD = result
    
    out_ds = xr.Dataset({
                'eff': efficiency, 
                'diseq': diseq,
                'lnpi': lnpi,
                'lneff': lneff,
                'lndiseq': lndiseq,
                'lnCKCD': lnCKCD[0,0,0]
            })
    
    # add names and units (where applicable)
    out_ds.eff.attrs['standard_name'],out_ds.eff.attrs['units']='Tropical Cyclone Efficiency','unitless fraction'
    out_ds.diseq.attrs['standard_name'],out_ds.diseq.attrs['units']='Thermodynamic Disequilibrium','J/kg'
    out_ds.lnpi.attrs['standard_name']='Natural log(Potential Intensity)'
    out_ds.lneff.attrs['standard_name']='Natural log(Tropical Cyclone Efficiency)'
    out_ds.lndiseq.attrs['standard_name']='Natural log(Thermodynamic Disequilibrium)'
    out_ds.lnCKCD.attrs['standard_name'],out_ds.lnCKCD.attrs['units']='Natural log(Ck/CD)','unitless constant'

    # return the output from pi.py as an xarray data structure
    return out_ds
    
    

if __name__ == "__main__":

    # Execute PI analysis over the whole dataset and save the output
    print('Beginning PI computations...')
    ds = run_sample_dataset(_FN)
    ds.to_netcdf(datdir+'raw_sample_output.nc')
    print('...PI computation complete and saved\n')
    
    # Perform PI analyses over the whole dataset
    print('Performing PI analyses...')
    ds2 = run_sample_analyses(ds,_mdrF,CKCD=0.9)
    
    # merge the arrays and save the output
    ds3=ds.merge(ds2)
    ds3.to_netcdf(datdir+'full_sample_output.nc')
    del ds, ds2
    print('...PI analyses complete and saved')
    print('pyPI sample run finished!')

# Potential intensity for a climate model

In [232]:
import glob
import warnings; warnings.filterwarnings("ignore", category = FutureWarning)
import xarray as xr
import xesmf as xe
import numpy as np

import matplotlib.pyplot as plt

## Regrid tos from unstructured to structured grid

Suggested by Nathan: https://stackoverflow.com/questions/3864899/resampling-irregularly-spaced-data-to-a-regular-grid-in-python  
Or using xesmf: https://pavics-sdi.readthedocs.io/en/latest/notebooks/regridding.html


In [233]:
tos = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/synda_clair/data/CMIP6/CMCC-ESM2/tos/tos_Oday_CMCC-ESM2_historical_r1i1p1f1_gn_18500101-18591231.nc").sel(time = "1850")

# add CF attributes to allow regridding
tos.i.attrs['axis'] = 'X'
tos.j.attrs['axis'] = 'Y'

In [237]:
# regrid SSTs to regular land-sea mask grid
rg = xe.Regridder(tos, lsm, "bilinear", ignore_degenerate = True)
tos2 = rg(tos)

/rds/general/user/cb2714/home/anaconda3/envs/wwa_xesmf/lib/python3.10/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


In [241]:
# check for breaks in the longitude
if any(tos2.sum(["lat", "time"]).tos == 0):
    
    # fill in gaps by interpolating across longitudes
    xlon = (tos2.sum(["lat", "time"]).tos).argmin().values
    tos2.tos[:,:,xlon] = tos2.tos.isel(lon = slice(xlon-1, xlon+2)).mean("lon").transpose("time", "lat")

In [243]:
tos2.to_netcdf("data/tos-regridded.nc")

## Compute PI from gridded data

In [1]:
import xarray as xr
import warnings; warnings.filterwarnings("ignore", category = FutureWarning)

from tcpyPI import pi
from tcpyPI.utilities import *

In [3]:
test_ds = xr.open_dataset("data/sample_data.nc")
test_ds

<xarray.Dataset>
Dimensions:  (p: 31, lat: 73, lon: 144, month: 12)
Coordinates:
  * p        (p) float64 1e+03 975.0 950.0 925.0 900.0 ... 40.0 30.0 20.0 10.0
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * month    (month) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0
Data variables:
    lsm      (lat, lon) float64 ...
    sst      (month, lat, lon) float64 ...
    msl      (month, lat, lon) float64 ...
    t        (month, p, lat, lon) float64 ...
    q        (month, p, lat, lon) float64 ...
    Vmax     (month, lat, lon) float64 ...
    To       (month, lat, lon) float64 ...
    Pmin     (month, lat, lon) float64 ...
    LNB      (month, lat, lon) float64 ...
    PI_flag  (month, lat, lon) float64 ...

In [2]:
# load each variable for a single year
hus = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/synda_clair/data/CMIP6/CMCC-ESM2/hus/hus_day_CMCC-ESM2_historical_r1i1p1f1_gn_18500101-18511231.nc").sel(time = "1850").hus
ta = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/synda_clair/data/CMIP6/CMCC-ESM2/ta/ta_day_CMCC-ESM2_historical_r1i1p1f1_gn_18500101-18511231.nc").sel(time = "1850").ta
psl = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/synda_clair/data/CMIP6/CMCC-ESM2/psl/psl_day_CMCC-ESM2_historical_r1i1p1f1_gn_18500101-18741231.nc").sel(time = "1850").psl
tos = xr.open_dataset("data/tos-regridded.nc").tos

lf = xr.open_dataset("data/sftlf_fx_CMCC-ESM2_historical_r1i1p1f1_gn.nc").sftlf
lsm = xr.ones_like(lf).where(lf >= 0.5, 0).rename("lsm")

In [3]:
ds = xr.merge([lsm, tos, psl, ta, hus]).rename(plev = "p", ta = "t", hus = "q", psl = "msl", tos = "sst")

In [5]:
result = xr.apply_ufunc(
    pi,
    ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
    kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),
    input_core_dims=[
        [], [], ['p', ], ['p', ], ['p', ],
    ],
    output_core_dims=[
        [], [], [], [], []
    ],
    vectorize=True
)

In [11]:
# store the result in an xarray data structure
vmax, pmin, ifl, t0, otl = result
out_ds=xr.Dataset({
    'vmax': vmax, 
    'pmin': pmin,
    'ifl': ifl,
    't0': t0,
    'otl': otl,
    })

In [13]:
out_ds.to_netcdf("PI.nc")